# Assessing spatial characteristics of informal settlement in Nairobi

## Imports

In [ ]:
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt

## Data Imports

1. Sentinel 2 L1C
2. OSM Boundaries
3. Slum Boundaries
4. Google Building Footprints

## Loading the slum boundaries

In [ ]:
gdf_slum = gpd.read_file("nairobi/slum-boundaries/slums_Nrb.shp")
gdf_slum.head()

Checking the CRS and attributes in the shapefile

In [ ]:
print("CRS of the slum settlement shapefile", gdf_slum.crs)

In [ ]:
print("Attributes")
print(gdf_slum.columns)

Calculating Area of Slum Settlements

In [ ]:
gdf_slum["area_acre"] = gdf_slum["geometry"].area / 4047


Selecting the biggest 10 landfill settlement

In [ ]:
gdf_slum = gdf_slum.loc[gdf_slum["area_acre"].nlargest(10).index]

## Loading the OSM buildings layer

In [ ]:
gdf_osm_buildings = gpd.read_file("nairobi/osm/buildings.shp")
gdf_osm_buildings.head()

### Counting buildings within slum settlements

In [ ]:
# ensure building shapefile has the same crs as slums
gdf_osm_buildings_reproject = gdf_osm_buildings.to_crs(gdf_slum.crs)

# apply spatial inner join with within query to get list of buildings within each slum
# resuls in a dataframe with a new column to buildings called object id of slums
buildings_within_slums = gpd.sjoin(gdf_osm_buildings_reproject, gdf_slum[["OBJECTID", "geometry"]],
                how="inner",
                predicate="within",
                lsuffix="left",
                rsuffix="right")

# Group by slum ID and count the buildings
building_counts = buildings_within_slums.groupby("OBJECTID").size().reset_index(name="building_count")

# finally merge building counts with slums 
slums_with_counts = gdf_slum.merge(building_counts, on="OBJECTID", how="left")

Building footprint data can be too large to fit into memory, try loading data in chunks

In [ ]:
# Function to read buildings CSV in chunks and process each chunk
def count_buildings_in_slums(slums, buildings_shp, crs, chunk_size=100000):
    building_counts = gpd.GeoDataFrame()  # Initialize empty GeoDataFrame to store counts
    
    # Process buildings in chunks
    for chunk in gpd.read_file(buildings_shp, chunksize=chunk_size):
        # reproject the chunk to match slums crs
        chunk = chunk.to_crs(crs)
        
        # Spatial join to find buildings within each slum
        buildings_within_slums = gpd.sjoin(chunk, slums[["OBJECTID", "geometry"]], how="inner", predicate="within")
        
        # Count buildings per slum in the current chunk
        chunk_counts = buildings_within_slums.groupby("OBJECTID").size().reset_index(name="building_count")
        
        # Append to the main building counts DataFrame
        building_counts = pd.concat([building_counts, chunk_counts], ignore_index=True)
    
    # Aggregate counts across chunks
    building_counts = building_counts.groupby("OBJECTID").sum().reset_index()
    return building_counts

# Calculate building counts and merge with slums GeoDataFrame
buildings_shp = "nairobi/osm/buildings.shp"
building_counts = count_buildings_in_slums(gdf_slum, buildings_shp)
slums_with_counts = gdf_slum.merge(building_counts, on="OBJECTID", how="left")

# Fill NaNs and set building count as integer
slums_with_counts["building_count"] = slums_with_counts["building_count"].fillna(0).astype(int)

## Loading the OSM Points Layer

In [ ]:
points_gdf = gpd.read_file("nairobi/osm/points.shp")

### Proximity Analysis

Finding toilets within 500m of slum boundaries

In [ ]:
buffer_size = 500

toilets = points_gdf[points_gdf["type"]=="toilets"]

# Create a 500m buffer around each slum polygon
gdf_slum['geometry'] = gdf_slum['geometry'].buffer(buffer_size)

# Perform spatial join to find toilets within each buffered slum polygon
joined_gdf = gpd.sjoin(toilets, gdf_slum[["OBJECTID", "geometry"]], how='inner', predicate='within')

# Count the toilets for each slum polygon
counts = joined_gdf.groupby('OBJECTID').size()

# Add the counts back to the slum_gdf
gdf_slum[f"toilet_within_{buffer_size}m"] = gdf_slum.OBJECTID.map(counts).fillna(0).astype(int)

Run the same logic for bus stops, hospitals and schools

## Loading the OSM line layers

Calculating the closest source of water and it distance from slums

In [ ]:
waterways = gpd.read_file("nairobi/osm_data/waterways.shp")

if gdf_slum.crs != waterways.crs:
    waterways = waterways.to_crs(gdf_slum.crs)

# Initialize lists to store results
min_distances = []
nearest_waterway_types = []

# Iterate over each slum polygon
for slum in gdf_slum['geometry']:
    # Calculate distance to each waterway and store in a DataFrame
    distances = waterways.distance(slum)
    nearest_idx = distances.idxmin()  # Index of the nearest waterway
    
    # Get the minimum distance and type of the nearest waterway
    min_distances.append(distances[nearest_idx])
    nearest_waterway_types.append(waterways.loc[nearest_idx, 'type'])

# Add results to the slums GeoDataFrame
gdf_slum['min_distance_to_waterway'] = min_distances
gdf_slum['nearest_waterway_type'] = nearest_waterway_types

Run the same logic for roads, railways

## Raster Analysis

Calculating NDVI using Sentinel 2 bands

In [ ]:
import rasterio
import numpy as np

# Define paths to the Red and NIR band files
filepath = "nairobi/sentinel-2/"
red_band_path = f"{filepath}B04.jp2"  # Band 4
nir_band_path = f"{filepath}B08.jp2"  # Band 8


# Open the Red and NIR bands as raster files
with rasterio.open(red_band_path) as red_src:
    red = red_src.read(1).astype('float32')  # Read the Red band and convert to float32
    profile = red_src.profile  # Store metadata for writing the output NDVI

with rasterio.open(nir_band_path) as nir_src:
    nir = nir_src.read(1).astype('float32')  # Read the NIR band and convert to float32

# Calculate NDVI using the formula
ndvi = (nir - red) / (nir + red)

# Handle NaN values (optional, e.g., if there are divide-by-zero results)
ndvi = np.where(np.isfinite(ndvi), ndvi, 0)

# Update profile to store NDVI as a single-band raster with float32 data type
profile.update(dtype='float32', count=1, driver="GTiff")

# Save the NDVI to a new GeoTIFF file
ndvi_output_path = "nairobi/ndvi_output.tif"
with rasterio.open(ndvi_output_path, 'w', **profile) as dst:
    dst.write(ndvi, 1)

print("NDVI calculation complete. Output saved to", ndvi_output_path)

Calculating slope from elevation map

In [ ]:
from osgeo import gdal

# keep the elevation map in local projection for nairobi it's 32737
dem_path = "nairobi/elevation.tif"
slope_path = "nairobi/slope.tif"
ds = gdal.DemProcessing(dem_path, slope_path, processing="slope")
ds = None

## Zonal Stats

Calculating ndvi and slope gradient for each slum settlements

In [ ]:
import numpy as np
from rasterstats import zonal_stats

ndvi_raster_path = "nairobi/ndvi.tif"

# Load the slum polygons
slums = gdf_slum

print(slums.crs)
# Define the green cover threshold for NDVI
green_threshold = 0.3

def percentage_green_cover(x):
    # Remove NaN values from x
    x = x[~np.isnan(x)]
    # Calculate percentage of pixels above threshold
    return (x > 0.3).sum() / len(x) * 100 if len(x) > 0 else 0


# Calculate zonal statistics for NDVI within each polygon, using a categorical threshold to count green cover pixels
stats = zonal_stats(
    slums,
    ndvi_raster_path,
    stats=["mean"],  # Calculate mean NDVI (optional, for reference)
    add_stats={'green_cover': percentage_green_cover},
    geojson_out=True
)

# Convert the zonal statistics output to a GeoDataFrame
slums_with_ndvi = gpd.GeoDataFrame.from_features(stats)

# Rename columns for clarity
slums_with_ndvi = slums_with_ndvi.rename(columns={"mean": "mean_ndvi", "green_cover": "percent_green_cover"})

# Convert green cover fraction to percentage
slums_with_ndvi["percent_green_cover"] = slums_with_ndvi["percent_green_cover"] * 100

# Display the result
print(slums_with_ndvi[["mean_ndvi", "percent_green_cover", "geometry"]].head())

Run the same logic for slope map